## Prepare

In [1]:
import os
import sys

import openai
from dotenv import load_dotenv, find_dotenv

sys.path.append('../..')

# read local .env file
_ = load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']

## Agent

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=chat)
agent = initialize_agent(tools, chat, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
# agent.run("What will be the weather in Shanghai three days from now?")
agent.run("三天后上海的天气会怎么样？")



> Entering new AgentExecutor chain...
The user is asking about the weather in Shanghai three days from now. I need to look up the current weather forecast for Shanghai.
Action: Search
Action Input: Weather forecast for Shanghai on the third day from now
Observation: ['Shanghai Extended Forecast with high and low temperatures ... Mostly sunny. Feels Like: 38 °F. Humidity: 34%. Precipitation: Rain: 0 Snow ...', 'in Shanghai fair, +19 °C. Weather. Maps News · China. Shanghai 3-Day Weather Forecast. Now Today Tomorrow. 3 days. 10 days Month', 'The Shanghai weather on September 30 night will be mixed with some thunderstorm clouds possible and warm. Chance of precipitation 20%, normal atmospheric ...', "TONIGHT'S WEATHER FORECAST. 12/24. 30°Lo. RealFeel® 26°. Partly cloudy and cold · TOMORROW'S WEATHER FORECAST. 12/25. 43°/ 32°. RealFeel® 41°. Partly sunny and ...", 'Day by day forecast ; Today · 1° 33° · -3° 27° ; Saturday 23rd DecemberSat 23rd · 5° 42° · -4° 24° ; Sunday 24th DecemberSun

## 记忆组件

In [2]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        """
The following is a friendly conversation between a human and an AI.
The AI is talkative and provides lots of specific details from its context.
If the AI does not know the answer to a question, it truthfully says it does not know.        
        """
    ),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}"),
])

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

conversation.predict(input="你好，我是程序员")

'你好！很高兴认识你。作为一个AI助手，我可以回答你的问题、提供信息和帮助解决问题。请随时告诉我需要什么帮助。\n\n你说你是程序员，那么你最常用的编程语言是哪个？我知道很多程序员都喜欢使用Python或JavaScript，因为它们有丰富的库和框架支持。但是，根据你的需求和偏好，也可能会选择其他语言。\n\n如果你遇到任何问题或困难，请告诉我，我将尽力提供帮助。'

## LangChain 表达式

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    "tell me a joke about {topic}"
)
model = ChatOpenAI(temperature=0)
output_parser = StrOutputParser()
chain = prompt | model | output_parser

response = chain.invoke({"topic": "bear"})
print(response)

Sure, here's one: Why don't bears use email? Because they can't handle too many attachments! How was that?


In [4]:
functions = [
    {
        "name": "joke",
        "description": "tell me a joke",
        "parameters": {
            "topic": {
                "type": "object",
                "properties": {
                    "setup": {
                        "type": "string",
                        "description": "The setup for the joke"
                    },
                    "punchline": {
                        "type": "string",
                        "description": "The punchline for the joke"
                    }
                },
                "required": ["setup", "punchline"]
            }
        }
    }
]

chain = prompt | model.bind(function_call={"name": "joke"}, functions=functions)
response = chain.invoke({"topic": "bear"}, config={})
print(response)

content="Sure, why not? Here's one for you: Why do bears hate shoes so much? Because they like to run around in their bear feet! Is that the kind of joke you were looking for?"


## 多功能组合链

In [5]:
from operator import itemgetter

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template("what country is the city {city} in? respond in {language}")
chain1 = prompt1 | model | StrOutputParser()
response = chain1.invoke({"person": "obama"})
print(response)
chain2 = (
        {"city": chain1, "language": itemgetter("language")}
        | prompt2
        | model
        | StrOutputParser()
)
chain2.invoke({"person": "obama", "language": "Chinese"})

Barack Obama was born in Honolulu, Hawaii.


'这个城市位于夏威夷州，美国。中文：这个城市位于夏威夷州，美国。'

In [8]:
from langchain.schema.runnable import RunnableMap

prompt1 = ChatPromptTemplate.from_template("generate a random color")
prompt2 = ChatPromptTemplate.from_template("what is a fruit of color: {color}")
prompt3 = ChatPromptTemplate.from_template("what is countries flag that has the color: {color}")
prompt4 = ChatPromptTemplate.from_template("what is the color of {fruit} and {country}")

chain1 = prompt1 | model | StrOutputParser()
chain2 = RunnableMap(color={"color": chain1}) | {"fruit": prompt2 | model | StrOutputParser(),
                                                 "country": prompt3 | model | StrOutputParser(), } | prompt4
chain2.invoke({})

ChatPromptValue(messages=[HumanMessage(content="what is the color of The name of the fruit that is this color is not something I can determine as it's a random color and not associated with any specific fruit. However, some fruits that are shades of purple include eggplant, acai berries, blackberries, and plums. and The country with a flag closest to the color you provided (#8A2BE2) is Dominica. However, please note that no country has an exact match for this specific shade of purple.")])